In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('..')

In [2]:
from priceanalytics.data import download_df_map, cache_df_map, load_cached_df_map
from priceanalytics.plot import MultiPlot
from priceanalytics import indicators as I
from priceanalytics.backtest import Backtester

from datetime import datetime, date, time

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


In [3]:
tickers = ['AAPL', 'MSFT']
raw_tdfs = download_df_map(tickers, interval=1)
# cache_df_map(raw_tdfs, _dir="cache")


raw AAPL: 25078 datapoints from 2023-10-18 to 2023-12-08
raw MSFT: 23010 datapoints from 2023-10-18 to 2023-12-08


In [48]:
# old_tdfs = load_cached_df_map(_dir="cache") # was @ interval = 5
# old_dataset = old_tdfs['AAPL'] + old_tdfs['MSFT']

tickers = ['AAPL', 'MSFT', 'AMZN']
new_tdfs = download_df_map(tickers, interval=1)
new_dataset = []
for ticker, dfs in new_tdfs.items():
    new_dataset += dfs
    
#dataset = old_dataset + new_dataset
dataset = new_dataset

datapts = sum([len(df) for df in dataset])
print("Loaded", len(dataset), "datasets containing", datapts, "datapoints")

raw AAPL: 24133 datapoints from 2023-10-20 to 2023-12-09
raw MSFT: 22223 datapoints from 2023-10-20 to 2023-12-09
raw AMZN: 24798 datapoints from 2023-10-20 to 2023-12-09
Loaded 108 datasets containing 70731 datapoints


In [108]:
import priceanalytics.backtest as bt
 
def test_and_strat(total, _len, sig_len, stop_loss=-10): #-1000% will never happen (hopefully)
    tpr = 1
    incrs = []
    
    buys, sells = [], []
    for n, df in enumerate(total):

        df = I.add_square_bounds_osc(df, _len=_len, sig_len=sig_len)
        df = df.iloc[_len:]

        with bt.Backtester(df, verbose=False) as b:
            last_buy = 0
            for i, fr in b:
                
                if b.crossover("sbosc_bull", "sbosc_bear"):
                    if not b.in_position: #should probably do this check automatically
                        b.buy()
                        last_buy = b.price()
                
                elif b.crossover("sbosc_signal", "sbosc_bull"):
                    if b.in_position: #should also probably do this check automatically
                        b.sell()
                    
                if b.in_position: # stop loss calculation
                    fallen = (b.price() - last_buy) / last_buy
                    if fallen < stop_loss:
                        b.sell()
                        inpos=False
                
                if b.in_position:
                   if i >= len(df) - 1:
                       b.sell()
            
            buys.append(b.buys)
            sells.append(b.sells)
            
            incr = b.results(1)
            incrs.append(incr)
            tpr *= (1 + incr)
    return tpr - 1, incrs, (buys, sells)


# important notes re. infrastructure

1. backtest should do calculations every loop -- there should be a function for this
2. need much more data -- def. rewrite data library
3. 

In [120]:
# JPM
new_new_tdfs = download_df_map(['NVDA'], interval=1)
validation_dataset = []
for ticker, dfs in new_new_tdfs.items():
    validation_dataset += dfs

raw NVDA: 27885 datapoints from 2023-10-23 to 2023-12-12


In [125]:
from priceanalytics.data import combine_dataset, calc_returns_homogenous

validation_dataset = [df for df in validation_dataset if len(df) > 0]
print('operated', len(validation_dataset), 'days')

print('initial', validation_dataset[0].index[0][1], 'at price', '$' + str(validation_dataset[0]['close'].iloc[0]))
print('final  ', validation_dataset[-1].index[-1][1], 'at price', '$' + str(validation_dataset[-1]['close'].iloc[-1]))

pr, incrs, trades = test_and_strat(validation_dataset, 26, 146, stop_loss=-.05) 
print('\nbought', sum([len(t) for t in trades[0]]), 'times')
print('sold', sum([len(t) for t in trades[1]]), 'times')

# for i, df in enumerate(validation_dataset):
#     df = I.add_square_bounds_osc(df, _len=14, sig_len=21)
#     df = df.iloc[40:] # [_len:]
#     buys, sells = trades[0][i], trades[1][i]
#     with MultiPlot(2, size=(18, 9)) as m:
#         print(f'returns {bt.perc_ret(incrs[i])}%')
#         m.ohlc(df)
#         #plt.scatter(buys.keys(), buys.values(), color="#00FF00", marker="^")
#         for b in buys.keys():  
#             plt.axvline(x=b, color="#00FF00")
#         #plt.scatter(sells.keys(), sells.values(), color="#FF0000", marker="v")
#         for s in sells.keys():
#             plt.axvline(x=s, color="#FF0000")
#         m.next()
        
#         plt.plot(np.array(df['sbosc_bull']), color='r')
#         plt.plot(np.array(df['sbosc_bear']), color='g')
#         plt.plot(np.array(df['sbosc_signal']), color='#000000')


print('')
bt.print_test_results(incrs)

operated 36 days
initial 2023-10-23 08:05:00+00:00 at price $413.54
final   2023-12-11 23:58:00+00:00 at price $464.84

bought 85 times
sold 85 times

Model Results

maximum:    5.84%
median:     0.33%
minimum:   -6.23%
mean:       0.23%
mode:       0.30%

operated:   36 days
returned:   7.58%


## notes

After a short time testing this out I noticed three things.
1. If neither line is above the signal line then it could be a choppy market.
2. If one line is almost on top of the signal line or barely above it then it could also be chop and sometimes it lasts a little while. Waiting until it is clearly above the signal line might work better.
3. If a position is taken and the bull/bear line goes below the signal line then it could be used as an exit. However it could also mean the market is going to have a bit of chop or slight retracement before continuing if the opposite line doesn't cross above the signal line.

So far it looks phenomenal

Oh I forgot to add, if in a position and the line stacks on top of the signal line then it can be an exit or everything else I listed in point 3 of my previous comment.